# Python






In [ ]:
import kdrag.parsers.sexp as sexp

type sexp = list | int | str
def interp_exp(env, s : sexp):
    match s:
        case int() | str():
            return s
        case ["+", *args]:
            return sum(interp_exp(env, arg) for arg in args)
        case ["-", a, b]:
            return interp_exp(env, a) - interp_exp(env, b)
        case ["-", x]:
            return -interp_exp(env, x)





In [ ]:
def compile_expr(env, dst, s : sexp):
    match s:
        case int() | str():
            return s
        case ["+", *args]:
            compiled_args = [compile_expr(arg) for arg in args]
            return f"({' + '.join(map(str, compiled_args))})"
        case ["-", a, b]:
            compiled_a = compile_expr(a)
            compiled_b = compile_expr(b)
            return f"({compiled_a} - {compiled_b})"
        case ["-", x]:
            compiled_x = compile_expr(x)
            return f"(-{compiled_x})"

# Old Racket
https://github.com/IUCompilerCourse/Essentials-of-Compilation/releases/download/racket-october-2023/book.pdf
https://github.com/IUCompilerCourse/Essentials-of-Compilation

In [31]:
%%file /tmp/ch1.rkt
#lang racket
(provide (all-defined-out))
;(module ch1 racket
(require rackunit)
(struct Int (value))
(struct Prim (op args))
(struct Prog (info body))

(define (leaf arith)
    (match arith
        [(Int v) #t]
        [(Prim 'read '()) #t]
        [(Prim '- (list e1)) #f]
        [(Prim '+ (list e1 e2)) #f]
        [(Prim '- (list e1 e2)) #f]
))

(check-true (leaf (Int 3)))
(check-true (leaf (Prim 'read '())))


(define (is_exp ast)
    (match ast
        [(Int v) #t]
        [(Prim 'read '()) #t]
        [(Prim '- (list e1)) (is_exp e1)]
        [(Prim '+ (list e1 e2)) (and (is_exp e1) (is_exp e2))]
        [(Prim '- (list e1 e2)) (and ( is_exp e1) (is_exp e2))]
        [else #f]
    )
)

(check-true (is_exp (Int 3)))
(check-true (is_exp (Prim 'read '())))
(check-true (is_exp (Prim '- (list (Int 3)))))
(check-true (is_exp (Prim '+ (list (Int 3) (Int 4)))))
(check-false (is_exp (Prim 'foo '())))

(define (is_Lint ast)
    (match ast
        [(Prog info body) (and (string? info) (is_exp body))]
        [else #f]
    )
)

(define (interp_exp e)
    (match e
        [(Int v) v]
        [(Prim 'read '()) (read)]
        [(Prim '- (list e1)) (- (interp_exp e1))]
        [(Prim '+ (list e1 e2)) (+ (interp_exp e1) (interp_exp e2))]
        [(Prim '- (list e1 e2)) (- (interp_exp e1) (interp_exp e2))]
        [else (error "unknown expression" e)]
    )
)
(print "load ch1")

;)

Overwriting /tmp/ch1.rkt


In [32]:
! racket /tmp/ch1.rkt

"load ch1"

In [33]:
%%file /tmp/imp.rkt
#lang racket
(require (file "/tmp/ch1.rkt"))
(is_exp (Int 3))

Overwriting /tmp/imp.rkt


In [34]:
! racket /tmp/imp.rkt

"load ch1"#t


https://docs.racket-lang.org/rackunit/api.html


Could do oleg ref cell
open functions. typeclasses + tagless final something


In [92]:
%%file /tmp/dt.rkt
#lang racket

(hash 'a 1 'b 2 'c 3)
(make-hash)
'#hash((a . 1) (b . 2) (c . 3))
(vector 1 2 3)
(vector-immutable 1 2 3)
(vector-length #(1 2 3))
(define v (vector 1 2 3))
(vector-set! v 0 10)
v
(vector->list v)
(vector-map (lambda (x) (* x x)) v)
(make-vector 2)
(set-mutable? (set 1 2 3 1)) ; not mutable by default
(mutable-set 1 2 3 1)

Overwriting /tmp/dt.rkt


In [93]:
! racket /tmp/dt.rkt

'#hash((a . 1) (b . 2) (c . 3))
'#hash()
'#hash((a . 1) (b . 2) (c . 3))
'#(1 2 3)
'#(1 2 3)
3
'#(10 2 3)
'(10 2 3)
'#(100 4 9)
'#(0 0)
#f
(mutable-set 1 2 3)


```clojure
(define interp-Lint-class
    (class object%
        (super-new)
        (define/public ((interp_exp env) e)
            (match e
            [(Int n) n]
            [(Prim 'read '())
                (define r (read))
                (cond [(fixnum? r) r]
                    [else (error 'interp_exp "expected an integer" r)])]
            [(Prim '- (list e)) (fx- 0 ((interp_exp env) e))]
            [(Prim '+ (list e1 e2))
                (fx+ ((interp_exp env) e1) ((interp_exp env) e2))]
            [(Prim '- (list e1 e2))
                (fx- ((interp_exp env) e1) ((interp_exp env) e2))])
        )
            
        
        (define/public (interp_program p)
            (match p
            [(Program '() e) ((interp_exp '()) e)]))
))
```

In [41]:
%%file /tmp/ch2.rkt
#lang racket

 (require racket/fixnum)
(require (file "/tmp/ch1.rkt"))

(define interp-Lint-class
    (class object%
        (super-new)
        (define/public ((interp_exp env) e)
            (match e
                [(Int n) n]
            [(Prim 'read '())
                (define r (read))
                (cond [(fixnum? r) r]
                    [else (error 'interp_exp "expected an integer" r)])]
            [(Prim '- (list e)) (fx- 0 ((interp_exp env) e))]
            [(Prim '+ (list e1 e2))
                (fx+ ((interp_exp env) e1) ((interp_exp env) e2))]
            [(Prim '- (list e1 e2))
                (fx- ((interp_exp env) e1) ((interp_exp env) e2))]))
        
        (define/public (interp_program p)
            (match p
            [(Program '() e) ((interp_exp '()) e)]))
))



Overwriting /tmp/ch2.rkt


In [42]:
! racket /tmp/ch2.rkt

/tmp/ch2.rkt:24:13: match: syntax error in pattern
  in: (Program (quote ()) e)
  location...:
   /tmp/ch2.rkt:24:13
  context...:
   /usr/racket/collects/racket/match/gen-match.rkt:56:11: mk
   /usr/racket/collects/racket/match/../../syntax/parse/private/parse-interp.rkt:643:50
   /usr/racket/collects/racket/match/gen-match.rkt:25:0: go
   /usr/racket/collects/racket/private/stxparam.rkt:63:2
   [repeats 3 more times]
   /usr/racket/collects/racket/private/class-undef.rkt:35:0


# Python
